In [ ]:
def encode_mlp(obj):
    if isinstance(obj, Layer):
        return { "neurons": obj.neurons }
    elif isinstance(obj, Neuron):
        weights = [{ "value": { "Value": { "id": v.id, "nid": v.cid, "data": v.data } }, "input_node_id": 0 } for v in obj.w]
        neuron = {
            "weights": weights,
            "bias": { "id": obj.b.id, "nid": obj.b.cid, "data": obj.b.data }
        }
        if obj.nonlin:
            neuron["nonlin"] = { "Expr": { "id": 0, "op": { "Relu": 0 } }}
        return neuron
    elif isinstance(obj, MLP):
        return {
            "layers": obj.layers,
        }

with open("../../mlp_init.json", "w") as f:
    f.write(json.dumps(model, default=encode_mlp, indent=2))
    
        

In [ ]:
# [[[w.data for w in n.w] for n in l.neurons] for l in model.layers]
def encode_graph(n):
    if isinstance(n, Value):
        if n.op == '':
            return { "Value": { "id": n.id, "cid": n.cid, "data": n.data } }
        op = ''
        operands = [v.id for v in n.prev]
        if len(n.prev) == 1:
            operands = n.prev[0].id
        if n.op == "+":
            op = "Add"
        elif n.op == "sum":
            op = 'Sum'
        elif n.op == "*":
            op = "Mul"
        elif n.op == "ReLU":
            op = "Relu"
        elif n.op.startswith("**"):
            operands.push(n.op[2:])
            op = "Pow"
        cur = { "Expr": { "id": n.id, "cid": n.cid, "op": { op: operands }}}
        return cur

visited = set()
topo = []
def build_topo(n):
    if not n in visited:
        visited.add(n)
        for child in n.prev:
            build_topo(child)
        topo.append(n)

build_topo(total_loss)

with open("../../graph_init.json", "w") as f:
    f.write(json.dumps(topo, default=encode_graph, indent=1))
    
        